Inicialização do navegador

In [1]:
# criando navegador
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

servico = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=servico)

importando a base de dados

In [2]:
# importando base de dados(produtos a pesquisar)
tabela = pd.read_excel('buscas.xlsx')


Deifinição das funções de busca do google shopping e do buscape

In [3]:
import time
def verifica_se_tem_termos_banidos(lista_termos_banidos, nome ):
    tem_temos_banidos = False
    for palavra in lista_termos_banidos:
        if palavra in nome:
            tem_temos_banidos = True
    return tem_temos_banidos

def verifica_se_tem_todos_termos_produtos(lista_produtos, nome):
    tem_todos_termos_produto = True
    for palavra in lista_produtos:
        if palavra not in lista_produtos:
            tem_todos_termos_produto = False
    return  tem_todos_termos_produto



def buscar_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo): # parametros
    produto = produto.lower()
    lista_produtos = produto.split(' ')
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    lista_ofertas = []
    # Entra no google
    driver.get('https://www.google.com/')
    driver.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto, Keys.ENTER) # modulo keys para usar uma tecla
    #entrando na aba shopping
    driver.find_element(By.XPATH, '//*[@id="hdtb-sc"]/div/div/div[1]/div/div[2]/a/div').click()

    # pegando um elemento(valor, texto, produto)
    lista_resultados = driver.find_elements('class name', 'i0X6df')

    for resultado in lista_resultados:
        nome = resultado.find_element('class name', 'tAxDx').text
        nome = nome.lower()
        # analisando se tem termo banido
        tem_termos_banidos = verifica_se_tem_termos_banidos(lista_termos_banidos, nome)
        # analisando se tem todos os elementos do produto        
        tem_todos_termos_produto = verifica_se_tem_todos_termos_produtos(lista_produtos, nome)
        # selecionar só os elementos que tem_termos_banidos = False e os msm tempo, tem_todos_termos_produtos = True       
        if tem_termos_banidos == False and tem_todos_termos_produto == True:
            preco = resultado.find_element('class name', 'a8Pemb').text # não estamos pesquisando no navegador inteiro, por isso, estamos procurando o item na variavel resultado, que vai iterar os valores da variavel lista_resultados
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.').replace('+impostos', '')
            
            preco = float(preco)
            # se o preço ta entre o preço minimo e preeço maxx
            if preco_minimo <= preco <= preco_maximo:
                elemento_referencia = resultado.find_element('class name', 'bONr3b') # elemento filho(referencia)
                elemento_pai = elemento_referencia.find_element('xpath', '..') # pegando o elemento pai, acima do elemento filho
                link = elemento_pai.get_attribute('href') # atravez do elemento pai, pegamos o seu HREF
                lista_ofertas.append(( nome, preco, link))
    return lista_ofertas     


def busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo):
    #tratar os valores
    produto = produto.lower()
    lista_produtos = produto.split(' ')
    termos_banidos = termos_banidos.lower()
    lista_termos_banidos = termos_banidos.split(' ')
    lista_ofertas = []
    #buscar no buscape
    driver.get('https://www.buscape.com.br/')
    driver.find_element('xpath', '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto, Keys.ENTER)
    #analisar se o resultado tem o termo banido e tem todos os termos do produto
    time.sleep(5)
    lista_resultados = driver.find_elements('class name', 'ProductCard_ProductCard_Inner__gapsh')
    for resultado in lista_resultados:
        nome = resultado.find_element('class name', 'ProductCard_ProductCard_Name__U_mUQ').text
        nome.lower()
        # analisando se tem termo banido
        tem_termos_banidos = verifica_se_tem_termos_banidos(lista_termos_banidos, nome)
        # analisando se tem todos os elementos do produto        
        tem_todos_termos_produto = verifica_se_tem_todos_termos_produtos(lista_produtos, nome)
        # caso o item, esteja de acordo com a condição, o preco é tratado
        if tem_termos_banidos == False and tem_todos_termos_produto == True:
            preco = resultado.find_element('class name', 'Text_MobileHeadingS__HEz7L').text
            preco = preco.replace('R$', '').replace(' ', '').replace('.', '').replace(',', '.')
            preco = float(preco)
            #verificando se o valor do prod está entre os valores informados 
            if preco_minimo <= preco <= preco_maximo:
                link = resultado.get_attribute('href')
                lista_ofertas.append((nome, preco, link ))
    return lista_ofertas

    # analisar se o valor está entro o min e o maxx
    #retornas a lista



Construindo a lista de ofertas

In [4]:

tabela_ofertas = pd.DataFrame()



for linha in tabela.index:
    produto = tabela.loc[linha, 'Nome']
    termos_banidos = tabela.loc[linha, 'Termos banidos']
    preco_minimo = tabela.loc[linha, 'Preço mínimo']
    preco_maximo = tabela.loc[linha, 'Preço máximo']
    preco_minimo = float(preco_minimo)
    preco_maximo = float(preco_maximo)

    lista_ofertas_google_shooping = buscar_google_shopping(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    #verifica se a lista esté vazia
    if lista_ofertas_google_shooping:
        tabela_google_shopping = pd.DataFrame(lista_ofertas_google_shooping, columns=['produto', 'preco', 'link loja'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_google_shopping]) # concatenando a tabela(pd.concat)
    else: 
        tabela_google_shopping = None

    lista_ofertas_buscape = busca_buscape(driver, produto, termos_banidos, preco_minimo, preco_maximo)
    if lista_ofertas_buscape:
        tabela_buscape = pd.DataFrame(lista_ofertas_buscape, columns=['produto', 'preco', 'link loja'])
        tabela_ofertas = pd.concat([tabela_ofertas, tabela_buscape]) # concatenando a tabela(pd.concat)
    else: 
        tabela_buscape = None

print(tabela_ofertas)

                                              produto    preco  \
0                              storytelling com dados    73.12   
1   storytelling com dados: um guia sobre visualiz...    67.00   
2   storytelling com dados - edição colorida + mar...    68.65   
3            storytelling com dados - vamos praticar!    56.20   
4   storytelling: aprenda a contar histórias com s...    56.05   
..                                                ...      ...   
6              iphone 14 plus 128gb product (red) bom  3730.65   
7                          apple iphone 14 azul 128gb  3674.46   
8   apple iphone 14 pro max - 128gb 256gb 512gb 1t...  3565.60   
0   Smartphone Apple iPhone 14 Usado 256GB Câmera ...  3527.92   
1   Smartphone Apple iPhone 14 Usado 128GB Câmera ...  3619.00   

                                            link loja  
0   https://www.google.com/url?url=https://www.ama...  
1   https://www.google.com/url?url=https://dragons...  
2   https://www.google.com/url?url=http

exportando para excel

In [5]:
# exportar para excel
tabela_ofertas.to_excel('Ofertas..xlsx', index=False)


Enviando o email

In [101]:
# enviando email
import win32com.client as win32
#verificando se existe alguma oferta dentro da tabela de ofertas
if len(tabela_ofertas.index) > 0:
    # vou enviar email
    outlook = win32.Dispatch('outlook.application')
    mail = outlook.CreateItem(0)
    mail.To = 'victorgabrieltrabalhos@gmail.com'
    mail.Subject = 'Produto(s) Encontrado(s) na faixa de preço desejada'
    mail.HTMLBody = f"""
    <p>Prezado, Victor</p>
    <p>Encontrei alguns produtos em oferta dentro da faixa de preço desejada. Segue tabela com detalhes</p>
    {tabela_ofertas.to_html(index=False)}
    <p>Qualquer dúvida estou à disposição</p>
    <p>Att.,
    Mikael</p>
    """
    
    mail.Send()